In [9]:
#Data Collection

import requests
import pandas as pd
import time

SEARCH_URL = "https://openlibrary.org/search.json"
WORK_URL = "https://openlibrary.org/works/{}.json"


def fetch_work_details(work_id):
    """
    Fetch detailed info for a single work, including genres/subjects.
    """
    try:
        url = WORK_URL.format(work_id)
        response = requests.get(url)
        data = response.json()

        subjects = data.get("subjects", [])
        genres = ", ".join(subjects[:5])  # top 5 genres/subjects

        return genres

    except Exception:
        return ""


def fetch_books(query="fiction", pages=1):
    """
    Fetch books from Open Library search API.
    Then fetch subjects/genres separately.
    """
    books = []

    for page in range(1, pages + 1):
        print(f"Fetching page {page}...")

        params = {"q": query, "page": page, "limit": 100}
        response = requests.get(SEARCH_URL, params=params)
        data = response.json()

        for item in data.get("docs", []):
            title = item.get("title", "")
            authors = ", ".join(item.get("author_name", []))
            pub_year = item.get("first_publish_year")

            popularity = item.get("edition_count", 0)

            # Extract work ID for detailed lookup:
            # e.g. key: "/works/OL12345W"
            work_key = item.get("key", "")
            work_id = work_key.replace("/works/", "") if "/works/" in work_key else None

            # Fetch genres from Work API
            genres = fetch_work_details(work_id) if work_id else ""

            books.append({
                "Title": title,
                "Author": authors,
                "Genres": genres,
                "Publication Year": pub_year,
                "Popularity": popularity
            })

            time.sleep(0.2)  # polite delay for Work API

        time.sleep(1)  # delay for Search API

    return books


def create_rankings(df):
    df = df.sort_values(by="Popularity", ascending=False).reset_index(drop=True)
    df["Rank"] = df.index + 1
    return df


def save_data(df):
    df.to_csv("books_scraped.csv", index=False)
    df.to_excel("books_scraped.xlsx", index=False)
    print("✔ Saved: books_scraped.csv and books_scraped.xlsx")


if __name__ == "__main__":
    raw_books = fetch_books(query="best books", pages=2)
    df = pd.DataFrame(raw_books)
    df = create_rankings(df)
    save_data(df)
    print(df.head())


Fetching page 1...
Fetching page 2...
✔ Saved: books_scraped.csv and books_scraped.xlsx
                              Title         Author  \
0               Pride and Prejudice    Jane Austen   
1  Alice's Adventures in Wonderland  Lewis Carroll   
2                             Bible          Bible   
3        The Picture of Dorian Gray    Oscar Wilde   
4                 Wuthering Heights  Emily Brontë   

                                              Genres  Publication Year  \
0  Fiction, Romance, Historical, Regency, British...              1813   
1  Alice (fictitious character : carroll), fictio...              1865   
2  Manuscripts, Latin (Medieval and modern), Glos...              1200   
3  British and irish fiction (fictional works by ...              1890   
4  British and irish fiction (fictional works by ...              1846   

   Popularity  Rank  
0        4036     1  
1        3546     2  
2        3173     3  
3        3007     4  
4        2885     5  


In [1]:
#Data Processing
#load the data using Panda
import pandas as pd

df = pd.read_csv("books_scraped.csv")
df.head()


,Title,Author,Genres,Publication Year,Popularity,Rank
0,Pride and Prejudice,Jane Austen,"Fiction, Romance, Historical, Regency, British...",1813,4036,1
1,Alice's Adventures in Wonderland,Lewis Carroll,"Alice (fictitious character : carroll), fictio...",1865,3546,2
2,Bible,Bible,"Manuscripts, Latin (Medieval and modern), Glos...",1200,3173,3
3,The Picture of Dorian Gray,Oscar Wilde,British and irish fiction (fictional works by ...,1890,3007,4
4,Wuthering Heights,Emily Brontë,British and irish fiction (fictional works by ...,1846,2885,5


In [13]:
# Remove unwanted books
unwanted_keywords = [
    "notebook", "journal", "coloring", "study guide",
    "blank", "malbuch", "composition", "for dummies"
]

df = df[~df["Title"].str.lower().str.contains("|".join(unwanted_keywords))]
print(df)

                                Title            Author  \
0                 Pride and Prejudice       Jane Austen   
1    Alice's Adventures in Wonderland     Lewis Carroll   
2                               Bible             Bible   
3          The Picture of Dorian Gray       Oscar Wilde   
4                   Wuthering Heights     Emily Brontë   
..                                ...               ...   
166                    A single shard    Linda Sue Park   
167                    The Best of Me   Nicholas Sparks   
168   The Best Christmas Pageant Ever  Barbara Robinson   
180                    I Am an Artist        best books   
191                  Best Pre-K Books        Best Books   

                                                Genres  Publication Year  \
0    fiction, romance, historical, regency, british...              1813   
1    alice (fictitious character : carroll), fictio...              1865   
2    manuscripts, latin (medieval and modern), glos...         

In [14]:
# Clean the genre column
df["Genres"] = df["Genres"].str.lower().str.strip()
print(df["Genres"])


0      fiction, romance, historical, regency, british...
1      alice (fictitious character : carroll), fictio...
2      manuscripts, latin (medieval and modern), glos...
3      british and irish fiction (fictional works by ...
4      british and irish fiction (fictional works by ...
                             ...                        
166    pottery, newbery medal, fiction, orphans, juve...
167    new york times bestseller, first loves, fictio...
168    brothers and sisters, aunts, dysfunctional fam...
180                                                  NaN
191                                                  NaN
Name: Genres, Length: 167, dtype: object


In [16]:
# Save the cleaned data
df.to_csv("books_cleaned.csv", index=False)
df.to_excel("books_cleaned.xlsx", index=False)
print("✔ Saved cleaned files.")


✔ Saved cleaned files.


In [2]:
#User Interaction
#Filtered Book list

def filter_by_genre(df, genre):
    genre = genre.lower().strip()

    return df[df["Genres"].fillna("").apply(lambda g: genre in g.lower())]



def filter_by_popularity(df, min_popularity):
    return df[df["Popularity"] >= min_popularity]


def filter_by_rank(df, start_rank=None, end_rank=None):
    result = df.copy()
    if start_rank:
        result = result[result["Rank"] >= start_rank]
    if end_rank:
        result = result[result["Rank"] <= end_rank]
    return result


def filter_by_year(df, start_year=None, end_year=None):
    result = df.copy()
    if start_year:
        result = result[result["Publication Year"] >= start_year]
    if end_year:
        result = result[result["Publication Year"] <= end_year]
    return result


def filter_books(df, genre=None, min_popularity=None,
                 rank_start=None, rank_end=None,
                 year_start=None, year_end=None):

    result = df.copy()

    if genre:
        result = filter_by_genre(result, genre)

    if min_popularity:
        result = filter_by_popularity(result, min_popularity)

    if rank_start or rank_end:
        result = filter_by_rank(result, rank_start, rank_end)

    if year_start or year_end:
        result = filter_by_year(result, year_start, year_end)

    return result


In [3]:
def user_filter_interface(df):
    print("\n=== Book Filtering Tool ===")

    genre = input("Genre (leave blank to skip): ").strip()
    genre = genre if genre else None

    min_popularity = input("Minimum popularity (edition count) (blank to skip): ").strip()
    min_popularity = int(min_popularity) if min_popularity else None

    rank_start = input("Minimum rank (blank to skip): ").strip()
    rank_start = int(rank_start) if rank_start else None

    rank_end = input("Maximum rank (blank to skip): ").strip()
    rank_end = int(rank_end) if rank_end else None

    year_start = input("Start publication year (blank to skip): ").strip()
    year_start = int(year_start) if year_start else None

    year_end = input("End publication year (blank to skip): ").strip()
    year_end = int(year_end) if year_end else None

    result = filter_books(
        df,
        genre=genre,
        min_popularity=min_popularity,
        rank_start=rank_start,
        rank_end=rank_end,
        year_start=year_start,
        year_end=year_end
    )

    print("\n=== Filtered Results ===")
    print(result[["Rank", "Title", "Author", "Genres", "Publication Year", "Popularity"]])

    print(f"\nTotal books found: {len(result)}")
    return result


In [19]:
filtered=user_filter_interface(df)
print(filtered)



=== Book Filtering Tool ===


Genre (leave blank to skip):  religion
Minimum popularity (edition count) (blank to skip):  
Minimum rank (blank to skip):  
Maximum rank (blank to skip):  
Start publication year (blank to skip):  
End publication year (blank to skip):  



=== Filtered Results ===
   Rank  Title Author                                             Genres  \
2     3  Bible  Bible  Manuscripts, Latin (Medieval and modern), Glos...   

   Publication Year  Popularity  
2              1200        3173  

Total books found: 1
   Title Author                                             Genres  \
2  Bible  Bible  Manuscripts, Latin (Medieval and modern), Glos...   

   Publication Year  Popularity  Rank  
2              1200        3173     3  


In [22]:
#Random Book Suggestion

def random_book_suggestion(df):
    """
    Generate a truly random book suggestion from a user-selected genre.
    """

    print("\n=== 🎲 Random Book Suggestion ===")
    genre = input("Enter a genre: ").strip().lower()

    # Filter by genre
    filtered_df = df[
        df["Genres"]
        .fillna("")
        .str.lower()
        .str.contains(genre)
    ]

    if filtered_df.empty:
        print(f"\n❌ No books found for genre: {genre}")
        return None

    # Truly random selection
    book = filtered_df.sample(n=1).iloc[0]

    print("\n📚 Your Random Book Pick:")
    print(f"Title: {book['Title']}")
    print(f"Author: {book['Author']}")
    print(f"Genres: {book['Genres']}")
    print(f"Publication Year: {book['Publication Year']}")
    print(f"Popularity (Edition Count): {book['Popularity']}")
    print(f"Rank: {book['Rank']}")

    return book


In [25]:
df = pd.read_csv("books_cleaned.csv")
random_book_suggestion(df)



=== 🎲 Random Book Suggestion ===


Enter a genre:  horror



📚 Your Random Book Pick:
Title: Dracula
Author: Bram Stoker
Genres: english literature, fiction, horror, horror stories, horror tales
Publication Year: 1897
Popularity (Edition Count): 736
Rank: 47


Title                                                         Dracula
Author                                                    Bram Stoker
Genres              english literature, fiction, horror, horror st...
Publication Year                                                 1897
Popularity                                                        736
Rank                                                               47
Name: 45, dtype: object